In [40]:
import lightgbm as lgbm

from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import optuna

In [17]:
train = pd.read_csv('preprocessed_train.csv', encoding='euc-kr')
test = pd.read_csv('preprocessed_test.csv', encoding='euc-kr')

In [18]:
train = train.drop(columns=['분석데이터'], axis=1)
test = test.drop(columns=['분석대상'], axis=1)

In [19]:
def summarize_categoricals(df, show_levels=False):
    """
        Display uniqueness in each column
    """
    data = [[df[c].unique(), len(df[c].unique()), df[c].isnull().sum()] for c in df.columns]
    df_temp = pd.DataFrame(data, index=df.columns,
                           columns=['Levels', 'No. of Levels', 'No. of Missing Values'])
    return df_temp.iloc[:, 0 if show_levels else 1:]


def to_categorical(columns, df):
    """
        Converts the columns passed in `columns` to categorical datatype
    """
    for col in columns:
        df[col] = df[col].astype('category')
    return df

In [20]:
summarize_categoricals(train, show_levels=True)

,Levels,No. of Levels,No. of Missing Values
label,"[1, 0]",2,0
numstrings,"[144, 804, 2205, 2602, 8980, 3312, 21761, 117,...",4830,0
avlength,"[12.29861111111111, 9.58084577114428, 12.73605...",8543,0
printables,"[1771, 7703, 28083, 26770, 208806, 82599, 1274...",7878,0
entropy,"[5.356616020202637, 6.063542366027832, 6.10704...",9494,0
...,...,...,...
dist_91,"[0, 36, 148, 64, 123, 997, 57, 7, 897, 23, 67,...",1232,0
dist_92,"[1, 40, 154, 26, 327, 181, 1134, 0, 139, 18, 1...",1345,0
dist_93,"[0, 45, 37, 702, 84, 100, 852, 23, 8, 759, 10,...",1224,0
dist_94,"[0, 27, 48, 1, 75, 1141, 62, 8, 1209, 76, 281,...",1249,0


In [21]:
train = to_categorical([], train)

In [22]:
x = train.iloc[:, 1:]

In [23]:
y = train['label']

In [24]:
categorical_columns = list(x.select_dtypes(include='category').columns)
numeric_columns = list(x.select_dtypes(exclude='category').columns)

In [59]:
from sklearn.model_selection import train_test_split

data_splits = train_test_split(x, y, test_size=0.2, random_state=42,
                               shuffle=True, stratify=y)
x_train, x_test, y_train, y_test = data_splits

list(map(lambda x: x.shape, [x, y, x_train, x_test, y_train, y_test]))

[(10000, 616), (10000,), (8000, 616), (2000, 616), (8000,), (2000,)]

In [185]:
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    param_grid = {


         'boosting':'gbdt',
         'metric': 'binary_logloss',
         'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3),
         'max_depth':trial.suggest_int("max_depth", 1, 50),
         'boosting':'gbdt',
         'metric': 'binary_logloss',
         'is_training_metric': True,
         'num_leaves':trial.suggest_int("num_leaves", 1, 50),
         'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 10, 1000, step=100),
         'feature_fraction':trial.suggest_float('feature_fraction', 0.2, 0.95, step=0.1),
         'bagging_fraction':trial.suggest_float('bagging_fraction', 0.2, 0.95, step=0.1),
         'bagging_freq':trial.suggest_categorical("bagging_freq", [5]),
         'alpha': trial.suggest_loguniform('alpha', 1e-3, 1.0),}

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)


In [171]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [172]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, x, y)
study.optimize(func, n_trials=20)

[I 2021-10-12 19:04:57,826] A new study created in memory with name: LGBM Classifier


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[1]	valid_0's binary_logloss: 0.567074
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.498488
[3]	valid_0's binary_logloss: 0.452988
[4]	valid_0's binary_logloss: 0.424724
[5]	valid_0's binary_logloss: 0.406705
[6]	valid_0's binary_logloss: 0.39218
[7]	valid_0's binary_logloss: 0.376474
[8]	valid_0's binary_logloss: 0.363436
[9]	valid_0's binary_logloss: 0.356196
[10]	vali

[73]	valid_0's binary_logloss: 0.438902
[74]	valid_0's binary_logloss: 0.433369
[75]	valid_0's binary_logloss: 0.432982
[76]	valid_0's binary_logloss: 0.430479
[77]	valid_0's binary_logloss: 0.429154
[78]	valid_0's binary_logloss: 0.428408
[79]	valid_0's binary_logloss: 0.429914
[80]	valid_0's binary_logloss: 0.430803
[81]	valid_0's binary_logloss: 0.445601
[82]	valid_0's binary_logloss: 0.463828
[83]	valid_0's binary_logloss: 0.475142
[84]	valid_0's binary_logloss: 0.472298
[85]	valid_0's binary_logloss: 0.479557
[86]	valid_0's binary_logloss: 0.57826
[87]	valid_0's binary_logloss: 0.572628
[88]	valid_0's binary_logloss: 0.56872
[89]	valid_0's binary_logloss: 0.582253
[90]	valid_0's binary_logloss: 0.589626
[91]	valid_0's binary_logloss: 0.596526
[92]	valid_0's binary_logloss: 0.635217
[93]	valid_0's binary_logloss: 0.629242
[94]	valid_0's binary_logloss: 0.654879
[95]	valid_0's binary_logloss: 0.680832
[96]	valid_0's binary_logloss: 0.934237
[97]	valid_0's binary_logloss: 0.963895
[9

[1]	valid_0's binary_logloss: 0.563402
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.499063
[3]	valid_0's binary_logloss: 0.454487
[4]	valid_0's binary_logloss: 0.429913
[5]	valid_0's binary_logloss: 0.408165
[6]	valid_0's binary_logloss: 0.396542
[7]	valid_0's binary_logloss: 0.385049
[8]	valid_0's binary_logloss: 0.38557
[9]	valid_0's binary_logloss: 0.375843
[10]	valid_0's binary_logloss: 0.372636
[11]	valid_0's binary_logloss: 0.370092
[12]	valid_0's binary_logloss: 0.358622
[13]	valid_0's binary_logloss: 0.355581
[14]	valid_0's binary_logloss: 0.354869
[15]	valid_0's binary_logloss: 0.351131
[16]	valid_0's binary_logloss: 0.351621
[17]	valid_0's binary_logloss: 0.350811
[18]	valid_0's binary_logloss: 0.346097
[19]	valid_0's binary_logloss: 0.344097
[20]	valid_0's binary_logloss: 0.34402
[21]	valid_0's binary_logloss: 0.345197
[22]	valid_0's binary_logloss: 0.342811
[23]	valid_0's binary_logloss: 0.348081
[24]	valid_0's binary_logloss

[74]	valid_0's binary_logloss: 0.483467
[75]	valid_0's binary_logloss: 0.485668
[76]	valid_0's binary_logloss: 0.492014
[77]	valid_0's binary_logloss: 0.485171
[78]	valid_0's binary_logloss: 0.488454
[79]	valid_0's binary_logloss: 0.490375
[80]	valid_0's binary_logloss: 0.488129
[81]	valid_0's binary_logloss: 0.488642
[82]	valid_0's binary_logloss: 0.487847
[83]	valid_0's binary_logloss: 0.489445
[84]	valid_0's binary_logloss: 0.48986
[85]	valid_0's binary_logloss: 0.491691
[86]	valid_0's binary_logloss: 0.569249
[87]	valid_0's binary_logloss: 0.575012
[88]	valid_0's binary_logloss: 0.575316
[89]	valid_0's binary_logloss: 0.57217
[90]	valid_0's binary_logloss: 0.575197
[91]	valid_0's binary_logloss: 0.893292
[92]	valid_0's binary_logloss: 0.914972
[93]	valid_0's binary_logloss: 0.934451
[94]	valid_0's binary_logloss: 1.36972
[95]	valid_0's binary_logloss: 1.0674
[96]	valid_0's binary_logloss: 1.73591
[97]	valid_0's binary_logloss: 0.943723
[98]	valid_0's binary_logloss: 0.96565
[99]	va

...

[I 2021-10-12 19:06:40,897] Trial 16 finished with value: 0.21902913841412314 and parameters: {'n_estimators': 3027, 'learning_rate': 0.26291476845410566, 'max_depth': 16, 'num_leaves': 15, 'min_data_in_leaf': 110, 'feature_fraction': 0.5, 'bagging_fraction': 0.7, 'bagging_freq': 5, 'alpha': 0.01219037767387841}. Best is trial 14 with value: 0.21365340323536489.


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=410, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=410
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


[I 2021-10-12 19:06:41,274] Trial 17 pruned. Trial was pruned at iteration 0.
[I 2021-10-12 19:06:41,676] Trial 18 pruned. Trial was pruned at iteration 1.


[1]	valid_0's binary_logloss: 0.597205
Training until validation scores don't improve for 100 rounds


[I 2021-10-12 19:06:42,062] Trial 19 pruned. Trial was pruned at iteration 0.


In [173]:
print(f"\tBest value (binary_logloss): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (binary_logloss): 0.21365
	Best params:
		n_estimators: 2687
		learning_rate: 0.2057841000706867
		max_depth: 19
		num_leaves: 20
		min_data_in_leaf: 110
		feature_fraction: 0.5
		bagging_fraction: 0.6000000000000001
		bagging_freq: 5
		alpha: 0.01646028003970267


In [186]:
train_ds = lgbm.Dataset(x_train, label = y_train)
test_ds = lgbm.Dataset(x_test, label = y_test)

'''params = {
        "n_estimators": 10000,
		"reg_lambda": 35,}'''

params = {
         'n_estimators': 5000,
         'learning_rate':0.2057841000706867,
         'max_depth':19,
         'boosting':'gbdt',
         'objective': 'binary',
         'metric': 'binary_logloss',
         'is_training_metric': True,
         'num_leaves':20,
         'min_data_in_leaf':110,
         'feature_fraction':0.5,
         'bagging_fraction':0.6000000000000001,
         'bagging_freq':5,
         'alpha': 0.01646028003970267,
         'seed':42}

model = lgbm.train(params, train_ds, 1000,test_ds, verbose_eval=100, early_stopping_rounds=100)
y_pred = model.predict(x_test)

[LightGBM] [Info] Number of positive: 4400, number of negative: 3600
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027533 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 151633
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 616
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.550000 -> initscore=0.200671
[LightGBM] [Info] Start training from score 0.200671
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.229762
[200]	valid_0's binary_logloss: 0.245217
Early stopping, best iteration is:
[100]	valid_0's binary_logloss: 0.229762


In [188]:
from sklearn.metrics import accuracy_score

for i in range(len(y_pred)):
    if y_pred[i]>=.5:
         y_pred[i]=1
    else:
        y_pred[i]=0

In [189]:
accuracy = accuracy_score(y_pred, y_test)

accuracy

0.897